In [2]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
from tqdm import tqdm

### 读入原始数据

In [2]:
folder_path = 'C:/Users/86355/Documents/py/628/hw3/climate'

In [3]:
# define every data type
dtype_dict = {
    "DepTime": "Int64",#9
    "ArrTime": "Int64",#13
    "CancellationCode": "category",#17
    "Origin_HourlyVisibility": "object",#41:"xxxx s""xxxx v"
    "Origin_HourlyWindDirection": "object",#43 catains VRB  #去掉？
    "Origin_HourlyWindSpeed": "object",#45 38.6s??? or float
    "Dest_HourlyPrecipitation": "object",#51 contains float and bool
    "Dest_HourlyPresentWeatherType": "object",#52 
    "Dest_HourlySkyConditions":"object",#56
    "Dest_HourlyVisibility": "object",#59:"xxxx s""xxxx v"
    "Dest_HourlyWindDirection": "object",#61 VBR
    "Dest_HourlyWindSpeed": "object"#63 38.6s ???
}

In [4]:
# 初始化一个空列表来存储每个 CSV 文件的 DataFrame
data_frames = []

# 使用 os.walk 递归遍历所有子文件夹和文件
for root, dirs, files in os.walk(folder_path):
    for file in files:
        # 检查文件是否为 CSV 文件
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            df = pd.read_csv(file_path, dtype=dtype_dict, na_values="NA", skipinitialspace=True)
            #df = pd.read_csv(file_path, na_values="NA", skipinitialspace=True)
            data_frames.append(df)
            #print(f"读取文件: {file}")

# 将所有 DataFrame 合并为一个总的 DataFrame
data_original = pd.concat(data_frames, ignore_index=True)

In [5]:
data_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9720184 entries, 0 to 9720183
Data columns (total 64 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   FlightDate                        object 
 1   Year                              int64  
 2   Month                             int64  
 3   DayofMonth                        int64  
 4   DayOfWeek                         int64  
 5   Operating_Airline                 object 
 6   Origin                            object 
 7   Dest                              object 
 8   CRSDepTime                        int64  
 9   DepTime                           Int64  
 10  DepDelay                          float64
 11  DepTimeBlk                        object 
 12  CRSArrTime                        int64  
 13  ArrTime                           Int64  
 14  ArrDelay                          float64
 15  ArrTimeBlk                        object 
 16  Cancelled                         in

In [6]:
# 计算每一列的缺失值比例
missing_ratio = data_original.isna().mean() * 100

# 循环输出每一列的缺失值比例
for column, ratio in missing_ratio.items():
    print(f"{column}: {ratio:.2f}%")

FlightDate: 0.00%
Year: 0.00%
Month: 0.00%
DayofMonth: 0.00%
DayOfWeek: 0.00%
Operating_Airline : 0.00%
Origin: 0.00%
Dest: 0.00%
CRSDepTime: 0.00%
DepTime: 1.92%
DepDelay: 1.92%
DepTimeBlk: 0.00%
CRSArrTime: 0.00%
ArrTime: 2.00%
ArrDelay: 2.16%
ArrTimeBlk: 0.00%
Cancelled: 0.00%
CancellationCode: 98.04%
Diverted: 0.00%
CRSElapsedTime: 0.00%
ActualElapsedTime: 2.16%
Distance: 0.00%
WeatherDelay: 82.66%
CRSDepDatetime: 0.00%
CRSDepTime_CST: 0.00%
DepTime_CST: 1.92%
ArrTime_CST: 2.16%
CRSArrTime_CST: 0.00%
Origin_REPORT_TYPE: 0.04%
Origin_SOURCE: 6.97%
Origin_HourlyAltimeterSetting: 21.44%
Origin_HourlyDewPointTemperature: 9.87%
Origin_HourlyDryBulbTemperature: 9.85%
Origin_HourlyPrecipitation: 26.62%
Origin_HourlyPresentWeatherType: 87.72%
Origin_HourlyPressureChange: 71.98%
Origin_HourlyPressureTendency: 71.98%
Origin_HourlyRelativeHumidity: 9.88%
Origin_HourlySkyConditions: 22.90%
Origin_HourlySeaLevelPressure: 16.24%
Origin_HourlyStationPressure: 12.86%
Origin_HourlyVisibility: 18.44

### 去掉Diverted==1

In [7]:
data_drop_Diverted = data_original[data_original['Diverted'] != 1]

In [8]:
data=data_drop_Diverted

### 根据我们能获取的天气信息删掉一些无法获取的变量，和diverted，和weathertype

In [9]:
# 列出要删除的列名
columns_to_drop = [
    'FlightDate', 'CRSDepTime', 'DepTime', 'CRSArrTime', 'ArrTime', 'CRSDepDatetime',
    'CRSDepTime_CST', 'DepTime_CST', 'ArrTime_CST', 'CRSArrTime_CST', 'Diverted','CRSElapsedTime',
    'Origin_HourlyPressureChange', 'Origin_HourlyPressureTendency', 'Origin_HourlySkyConditions',
    'Origin_HourlyStationPressure','Origin_HourlyWetBulbTemperature','Origin_HourlyPresentWeatherType',
    'Dest_HourlyPressureChange', 'Dest_HourlyPressureTendency', 'Dest_HourlySkyConditions',
    'Dest_HourlyStationPressure','Dest_HourlyWetBulbTemperature','Dest_HourlyPresentWeatherType',
    'CancellationCode','Origin_REPORT_TYPE','Origin_SOURCE','Dest_REPORT_TYPE','Dest_SOURCE','WeatherDelay'
]

# 从 data 中删除指定的列
data = data.drop(columns=columns_to_drop)

### 将一些变量转化为categroy

In [10]:
data['Year']=data['Year'].astype('category')
data['Month']=data['Month'].astype('category')
data['DayofMonth']=data['DayofMonth'].astype('category')
data['DayOfWeek']=data['DayOfWeek'].astype('category')
data['Operating_Airline ']=data['Operating_Airline '].astype('category')
data['Origin']=data['Origin'].astype('category')
data['Dest']=data['Dest'].astype('category')

注意'Operating_Airline '这个变量结尾有空格

### 将微量降水T转化为0.005

In [11]:
data['Origin_HourlyPrecipitation'] = data['Origin_HourlyPrecipitation'].replace('T', '0.005')
data['Origin_HourlyPrecipitation'] = pd.to_numeric(data['Origin_HourlyPrecipitation'], errors='coerce')
data['Dest_HourlyPrecipitation'] = data['Dest_HourlyPrecipitation'].replace('T', '0.005')
data['Dest_HourlyPrecipitation'] = pd.to_numeric(data['Dest_HourlyPrecipitation'], errors='coerce')

### 将风速中s结尾的去掉

In [12]:
data['Origin_HourlyWindSpeed'] = data['Origin_HourlyWindSpeed'].str.replace('s', '', regex=False)#去掉以s结尾
data['Origin_HourlyWindSpeed'] = pd.to_numeric(data['Origin_HourlyWindSpeed'], errors='coerce')
data['Dest_HourlyWindSpeed'] = data['Dest_HourlyWindSpeed'].str.replace('s', '', regex=False)#去掉以s结尾
data['Dest_HourlyWindSpeed'] = pd.to_numeric(data['Dest_HourlyWindSpeed'], errors='coerce')

### 去掉Visibility结尾的s和V，同时将*视为NA

In [13]:
data['Origin_HourlyVisibility'] = data['Origin_HourlyVisibility'].str.replace(r'(s|V)$', '', regex=True)
data['Origin_HourlyVisibility'] = pd.to_numeric(data['Origin_HourlyVisibility'], errors='coerce')
data['Dest_HourlyVisibility'] = data['Dest_HourlyVisibility'].str.replace(r'(s|V)$', '', regex=True)
data['Dest_HourlyVisibility'] = pd.to_numeric(data['Dest_HourlyVisibility'], errors='coerce')

### 将DepTimeBlk的时间区间转化为catagroy

In [14]:
data['DepTimeBlk'] = data['DepTimeBlk'].astype('category')
data['ArrTimeBlk'] = data['ArrTimeBlk'].astype('category')

In [15]:
#print(data['DepTimeBlk'].dtype)

### 将winddireaction以10为单位转化为category 同时去掉999 VBR改为361

先将字符串转化为浮点数

In [16]:
def convert_to_int(x):
    try:
        # 尝试将字符串转换为浮点数
        num = float(x)
        # 如果转换成功且不是 NaN，则返回整数
        return int(num) if not np.isnan(num) else np.nan
    except (ValueError, TypeError):
        # 如果不能转换为数值，则保持原值
        return x

data['Origin_HourlyWindDirection'] = data['Origin_HourlyWindDirection'].apply(convert_to_int)
data['Dest_HourlyWindDirection'] = data['Dest_HourlyWindDirection'].apply(convert_to_int)

In [17]:
# 将 winddirection 中为 float 类型的值转换为 int 类型（注意NAN不能转化为int）
data['Origin_HourlyWindDirection'] = data['Origin_HourlyWindDirection'].apply(lambda x: int(x) if isinstance(x, float) and not np.isnan(x) else x)
data['Dest_HourlyWindDirection'] = data['Dest_HourlyWindDirection'].apply(lambda x: int(x) if isinstance(x, float) and not np.isnan(x) else x)

In [18]:
# 定义合法的风向值集合（0 到 360，每隔 10）和VRB和999
valid_directions = list(range(0, 361, 10)) + ['VRB',999]

# 统计不在合法范围内的值
#invalid_values = data[~data['Origin_HourlyWindDirection'].isin(valid_directions)]['Origin_HourlyWindDirection']
#invalid_count = invalid_values.value_counts()
#print(invalid_count)
#invalid_values = data[~data['Dest_HourlyWindDirection'].isin(valid_directions)]['Dest_HourlyWindDirection']
#invalid_count = invalid_values.value_counts()
#print(invalid_count)

In [19]:
# 360 -> 0 'VRB'->361
data['Origin_HourlyWindDirection'] = data['Origin_HourlyWindDirection'].apply(
    lambda x: 0 if x == 360 else (361 if x == 'VRB' else (np.nan if x not in valid_directions else x))
)
data['Dest_HourlyWindDirection'] = data['Dest_HourlyWindDirection'].apply(
    lambda x: 0 if x == 360 else (361 if x == 'VRB' else (np.nan if x not in valid_directions else x))
)

# 将 999 设为 NaN
data['Origin_HourlyWindDirection'] = data['Origin_HourlyWindDirection'].replace(999, np.nan)
data['Dest_HourlyWindDirection'] = data['Dest_HourlyWindDirection'].replace(999, np.nan)

data['Origin_HourlyWindDirection'] = data['Origin_HourlyWindDirection'].astype('category')
data['Dest_HourlyWindDirection'] = data['Dest_HourlyWindDirection'].astype('category')

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9700884 entries, 0 to 9720183
Data columns (total 34 columns):
 #   Column                            Dtype   
---  ------                            -----   
 0   Year                              category
 1   Month                             category
 2   DayofMonth                        category
 3   DayOfWeek                         category
 4   Operating_Airline                 category
 5   Origin                            category
 6   Dest                              category
 7   DepDelay                          float64 
 8   DepTimeBlk                        category
 9   ArrDelay                          float64 
 10  ArrTimeBlk                        category
 11  Cancelled                         int64   
 12  ActualElapsedTime                 float64 
 13  Distance                          int64   
 14  Origin_HourlyAltimeterSetting     float64 
 15  Origin_HourlyDewPointTemperature  float64 
 16  Origin_HourlyDryBu

In [21]:
# 计算每一列的缺失值比例
missing_ratio = data.isna().mean() * 100

# 循环输出每一列的缺失值比例
for column, ratio in missing_ratio.items():
    print(f"{column}: {ratio:.2f}%")

Year: 0.00%
Month: 0.00%
DayofMonth: 0.00%
DayOfWeek: 0.00%
Operating_Airline : 0.00%
Origin: 0.00%
Dest: 0.00%
DepDelay: 1.92%
DepTimeBlk: 0.00%
ArrDelay: 1.97%
ArrTimeBlk: 0.00%
Cancelled: 0.00%
ActualElapsedTime: 1.97%
Distance: 0.00%
Origin_HourlyAltimeterSetting: 21.44%
Origin_HourlyDewPointTemperature: 9.87%
Origin_HourlyDryBulbTemperature: 9.85%
Origin_HourlyPrecipitation: 26.61%
Origin_HourlyRelativeHumidity: 9.88%
Origin_HourlySeaLevelPressure: 16.24%
Origin_HourlyVisibility: 18.44%
Origin_HourlyWindDirection: 19.92%
Origin_HourlyWindGustSpeed: 89.46%
Origin_HourlyWindSpeed: 9.85%
Dest_HourlyAltimeterSetting: 21.70%
Dest_HourlyDewPointTemperature: 9.72%
Dest_HourlyDryBulbTemperature: 9.70%
Dest_HourlyPrecipitation: 26.77%
Dest_HourlyRelativeHumidity: 9.74%
Dest_HourlySeaLevelPressure: 16.08%
Dest_HourlyVisibility: 18.46%
Dest_HourlyWindDirection: 19.90%
Dest_HourlyWindGustSpeed: 89.61%
Dest_HourlyWindSpeed: 9.70%


### 补充NA

对连续型变量我们采用条件均值对NA进行补充（因为直接补零是不合适的）

In [22]:
columns_to_fill = [
    'Origin_HourlyAltimeterSetting', 'Origin_HourlyDewPointTemperature', 
    'Origin_HourlyDryBulbTemperature', 'Origin_HourlyRelativeHumidity',
    'Origin_HourlySeaLevelPressure', 'Origin_HourlyVisibility', 'Origin_HourlyWindSpeed',
    'Dest_HourlyAltimeterSetting', 'Dest_HourlyDewPointTemperature', 
    'Dest_HourlyDryBulbTemperature', 'Dest_HourlyRelativeHumidity',
    'Dest_HourlySeaLevelPressure', 'Dest_HourlyVisibility', 'Dest_HourlyWindSpeed'
    
]

In [23]:
# Initialize a dictionary to store missing rates
missing_rates = {}

# Use tqdm to track the progress of filling each column
for column in tqdm(columns_to_fill, desc="Processing Columns"):
    # Step 1: Fill missing values based on a precise grouping (Year, Month, DayofMonth, Origin, DepTimeBlk)
    data[column] = data.groupby(['Year', 'Month', 'DayofMonth', 'Origin', 'DepTimeBlk'])[column].transform(lambda x: x.fillna(x.mean()))
    
    # Step 2: For any remaining missing values, use a broader grouping (Year, Month, DayofMonth) to fill them
    data[column] = data.groupby(['Year', 'Month', 'DayofMonth'])[column].transform(lambda x: x.fillna(x.mean()))
    
    # Calculate and print the missing rate for this column
    missing_rate = data[column].isna().mean() * 100
    missing_rates[column] = missing_rate
    print(f"Missing rate for {column} after processing: {missing_rate:.2f}%")

Processing Columns:   7%|████▍                                                         | 1/14 [04:04<53:01, 244.76s/it]

Missing rate for Origin_HourlyAltimeterSetting after processing: 0.00%


Processing Columns:  14%|████████▊                                                     | 2/14 [08:00<47:56, 239.73s/it]

Missing rate for Origin_HourlyDewPointTemperature after processing: 0.00%


Processing Columns:  21%|█████████████▎                                                | 3/14 [11:56<43:36, 237.85s/it]

...

Missing rate for Dest_HourlySeaLevelPressure after processing: 0.00%


Processing Columns:  93%|████████████████████████████████████████████████████████▋    | 13/14 [55:47<04:18, 258.65s/it]

Missing rate for Dest_HourlyVisibility after processing: 0.00%


Processing Columns: 100%|█████████████████████████████████████████████████████████████| 14/14 [59:54<00:00, 256.75s/it]

Missing rate for Dest_HourlyWindSpeed after processing: 0.00%


对于Precipitation和HourlyWindGustSpeed我们选择补0来代替NA

In [24]:
columns_to_fill_with_zero = [
    'Origin_HourlyPrecipitation', 'Origin_HourlyWindGustSpeed', 
    'Dest_HourlyPrecipitation', 'Dest_HourlyWindGustSpeed'
]

In [25]:
data[columns_to_fill_with_zero] = data[columns_to_fill_with_zero].fillna(0)

### 减少类别将日期按照节假日分为5个categories

In [26]:
tqdm.pandas()

# 定义分类函数，根据 Year、Month、DayofMonth 划分日期范围
def categorize_holiday(row):
    year = row['Year']
    date = datetime(year, row['Month'], row['DayofMonth'])
    
    # 动态计算感恩节日期（每年11月的第4个星期四）
    begin = datetime(year, 11, 1)
    thanksgiving = datetime(year, 11, 1) + pd.DateOffset(weeks=3) + pd.DateOffset(weekday=3)
    christmas = datetime(year, 12, 25)
    Newyear = datetime(year, 1, 1)
    
    # 按日期范围进行分类
    if begin <= date < thanksgiving - pd.DateOffset(days=1):
        return 'Before Thanksgiving'
    elif thanksgiving - pd.DateOffset(days=1) <= date <= thanksgiving + pd.DateOffset(days=4):
        return 'Around Thanksgiving'
    elif thanksgiving + pd.DateOffset(days=4) < date < christmas - pd.DateOffset(days=1):
        return 'Between Thanksgiving and Christmas'
    elif christmas - pd.DateOffset(days=1) <= date <= christmas + pd.DateOffset(days=1):
        return 'Around Christmas'
    elif christmas + pd.DateOffset(days=1) < date < Newyear - pd.DateOffset(days=1):
        return 'Between Christmas and Newyear'
    elif Newyear - pd.DateOffset(days=1) <= date <= Newyear + pd.DateOffset(days=1):
        return 'Around Newyear'
    else:
        return 'After Newyear'

data['Holiday'] = data.progress_apply(categorize_holiday, axis=1)

100%|██████████████████████████████████████████████████████████████████████| 9700884/9700884 [45:40<00:00, 3539.41it/s]


In [35]:
data['Holiday'] = data['Holiday'].astype('category')

In [39]:
data = data.drop('DayofMonth', axis=1)

### 处理winddirection中的NA

In [27]:
# 定义映射函数，根据给定的范围将值映射到新类别
def map_to_new_category(value):
    if pd.isna(value):
        return 'NA'
    elif value <= 40:
        return '0-40'
    elif 50 <= value <= 90:
        return '50-90'
    elif 90 < value <= 130:
        return '90-130'
    elif 140 <= value <= 180:
        return '140-180'
    elif 180 < value <= 220:
        return '180-220'
    elif 230 <= value <= 270:
        return '230-270'
    elif 270 < value <= 310:
        return '270-310'
    elif 320 <= value <= 350:
        return '320-350'
    elif value == 361:
        return '361'
    else:
        return 'NA'  # 默认返回NA以处理其他非标准值

# 应用映射函数创建新列
data['Origin_HourlyWindDirection'] = data['Origin_HourlyWindDirection'].apply(map_to_new_category)
data['Dest_HourlyWindDirection'] = data['Dest_HourlyWindDirection'].apply(map_to_new_category)

In [31]:
data['Origin_HourlyWindDirection'] = data['Origin_HourlyWindDirection'].fillna('Missing')
data['Dest_HourlyWindDirection'] = data['Dest_HourlyWindDirection'].fillna('Missing')

data['Origin_HourlyWindDirection'] = data['Origin_HourlyWindDirection'].astype('category')
data['Dest_HourlyWindDirection'] = data['Dest_HourlyWindDirection'].astype('category')

### 将洗好的数据存入csv

In [3]:
data=pd.read_csv('final_data.csv')

In [11]:
na_indices = data[data['Origin_HourlyWindSpeed'].isna()].index
data = data.drop(index=na_indices)

In [18]:
data.isna().sum()

Year                                     0
Month                                    0
DayOfWeek                                0
Operating_Airline                        0
Origin                                   0
Dest                                     0
DepDelay                            186393
DepTimeBlk                               0
ArrDelay                            190892
ArrTimeBlk                               0
Cancelled                                0
ActualElapsedTime                   190892
Distance                                 0
Origin_HourlyAltimeterSetting            0
Origin_HourlyDewPointTemperature         0
Origin_HourlyDryBulbTemperature          0
Origin_HourlyPrecipitation               0
Origin_HourlyRelativeHumidity            0
Origin_HourlySeaLevelPressure            0
Origin_HourlyVisibility                  0
Origin_HourlyWindDirection               0
Origin_HourlyWindGustSpeed               0
Origin_HourlyWindSpeed                   0
Dest_Hourly

In [17]:
data.to_csv('final_data.csv', index=False)

In [15]:
data['Holiday'] = data['Holiday'].astype('category')

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9700585 entries, 0 to 9700883
Data columns (total 34 columns):
 #   Column                            Dtype   
---  ------                            -----   
 0   Year                              int64   
 1   Month                             int64   
 2   DayOfWeek                         int64   
 3   Operating_Airline                 object  
 4   Origin                            object  
 5   Dest                              object  
 6   DepDelay                          float64 
 7   DepTimeBlk                        object  
 8   ArrDelay                          float64 
 9   ArrTimeBlk                        object  
 10  Cancelled                         int64   
 11  ActualElapsedTime                 float64 
 12  Distance                          int64   
 13  Origin_HourlyAltimeterSetting     float64 
 14  Origin_HourlyDewPointTemperature  float64 
 15  Origin_HourlyDryBulbTemperature   float64 
 16  Origin_HourlyPreci